**This notebook is the executable version of lab note 3.
It answers the following questions:**

Finally, we answer 4 items:

    1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?
    2. Are mavens more connected to successful creators than to unsuccessful creators?
    3. Do successful creators send more non-follow actions towards mavens than to zombies?
    4. Do successful creators send more non-follow actions towards mavens than to stars?

    

In [1]:
#Run parameters
#used to control every run. Can be user to perfom sensitivity checks
path_dir = r"/Users/../Volumes/Raw/"

low_success = 0.5 #below the median: unsuccessful
high_success = 0.9 #top 10% creators with more followers are deemed successful

low_user_outdegree = 0.25 
high_user_outdegree = 0.75
low_user_activity = 0.25 
high_user_activity = 0.75 

activity_filter = 0
days_delta = 7

In [2]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import os
from statsmodels.stats.proportion import proportions_ztest

In [3]:
def process_date(date):
    '''convert date format like '2013-w09' to '2013-03-04', i.e. the first day of that week'''
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.datetime.strptime(date, "%Y-%W-%w")
    return dt

# Data Imports

We start by importing the raw data.  `follows_sent`, `comments_sent`, `shares_sent`, `likes_sent` and `messages_sent` contains data pn the promotional activities that the 35k users tracked in the dataset directed to other users. It includes the `user_id`, the `fan_id` and the `date_sent` which identifies the date when the prom. activity was sent. `users_info_1st` shows the type of user (creator or non-creator, which is identified by a blank) and the date the user entered the platform, for every user that sent or received prom. activities from any of the 35k users tracked in this dataset, while `users_info` contains the same information, but pertaining to the 35k users themselves.

`follows_received` contains information on the follows received by the 35k users and will be used to generate the successful/unsuccessful groups of content creators.

In [4]:
#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#these are the actions sent to 
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)


Indegree and outdegree information.

The function below import the outdegree dataset. Because the raw version of those dataset are too large to be processed in memory, we preprocessed them in a separate script.

In [5]:
# Aggregates preprocessed outdegree of 1st degree users
def import_outdegree(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/'):
    d = {}
    for i in range(6):
       d[str(i)] = pd.read_pickle(os.path.join(path,'{}.pkl'.format(i))) 
       d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
       d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
       d[str(i)] = d[str(i)].groupby(['sender_id', 'created_at'], as_index = False).size() 
    
    data_outdegree = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
    #data_outdegree.set_index('created_at', inplace = True)
    return data_outdegree

In [6]:
# data_outdegree = import_outdegree()

KeyboardInterrupt: 

In [ ]:
#data_outdegree = data_outdegree.groupby(['sender_id','created_at'], as_index = False).size()

# Preprocessing

## Creator ids, successful and unsucessful creators

Next, we define three lists of ids: one with the ids from the content creators, according to the `users_info` table, one with the ids of successful creators and the last one with the ids of the unsuccessful ones.

Let's start with a list of the id of creators. We also create a dataset with containing information on creators only.

In [7]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

## Putting together a dataset with the promotional activities made by content creators.

The function `gen_actions_sent_df` creates a dataframe with all the promotional activities that content creators sent to users.

In [8]:
def gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent, messages_sent, creator_ids = creator_ids):
    '''
    Creates dataframe containing the actions that content creators send to users.
        Attributes:
                    follows_sent:  dataframe with the follows sent by the 35k users.
                    shares_sent:   dataframe with the shares sent by the 35k users.
                    likes_sent:    dataframe with the likes sent by the 35k users.
                    comments_sent: dataframe with the comments sent by the 35k users.
                    messages_sent: dataframe with the messages sent by the 35k users.
                    creator_ids:   list with content creator ids. If not none, is used to
                                   filter out activities from non creators.
    '''
    
    follows_sent['outbound_activity'] = 'follow'
    follows_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'song_id' in shares_sent.columns:
        shares_sent.drop(columns=["song_id"])
    shares_sent = shares_sent[['reposter_id', "owner_id", 'created_at']]
    shares_sent['outbound_activity'] = 'share'
    shares_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in likes_sent.columns:
        likes_sent.drop(columns=["track_id"], inplace=True)
    likes_sent['outbound_activity'] = 'like'
    likes_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in comments_sent.columns:
        comments_sent.drop(columns=["track_id"], inplace=True)
    comments_sent['outbound_activity'] = 'comment'
    comments_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    messages_sent["outbound_activity"] = 'message'
    messages_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']
    df = pd.concat([follows_sent, shares_sent, likes_sent, comments_sent, messages_sent])


    if type(creator_ids) == numpy.ndarray:
        df = df[df['user_id'].isin(creator_ids)]
        
    df['week_yr'] = df.date_sent.dt.strftime('%Y-w%U')
    df = df.loc[df['user_id'] != df['fan_id'],:]

    return df

In [9]:
actions_sent = gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent,
                                     messages_sent, creator_ids = None)
actions_sent = actions_sent.loc[actions_sent.user_id.isin(creators.user_id.unique())]

In [10]:
active_users_ids = actions_sent.groupby('user_id', as_index = False).size()
mask = active_users_ids['size']>= activity_filter
active_users_ids = active_users_ids[mask].user_id.unique()

In [11]:
def successful_creators_followers(follows_received, base_date = datetime.datetime(2016, 5, 30, 0, 0), perc1 = None, perc2 = None, subset_creators = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    follows_received: dataframe containing the follows received by content creators
                    base date:        date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                      of followers per creator is calculated.
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
                    subset_creators:  a pd.DataFrame containing the creators. If is it available, it will be used to 
                                      filter out non creators and to make sure creators with 0 followers are part of
                                      the resulting dataset.
        
    '''
    print(base_date)

    if 'inbound_activity' not in follows_received.columns:
        follows_received.columns = ['fan_id', 'user_id', 'date_sent']

    mask = (follows_received['date_sent'] < base_date)

    df = follows_received[mask].groupby('user_id', as_index=False).agg({'fan_id': pd.Series.nunique})
    df.columns = ['user_id', 'followers']

    
    if type(subset_creators) == pd.DataFrame:
        print('subsetting...')
        df.set_index('user_id', inplace = True)
        df = df.reindex(subset_creators.user_id.unique())
        df.fillna(0, inplace = True)
        df.reset_index(inplace = True)
        df.columns = ['user_id', 'followers']
        
    mask = df.user_id.isin(active_users_ids)
    df = df[mask]

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High influencer boundary: {}".format(high))
    print("Low influencer boundary: {}".format(low))

    mask = (df["followers"] <= low) | (df["followers"] >= high)
    
    unsuccessful_creator_ids = df.loc[df["followers"] <= low].user_id.unique()
    successful_creator_ids = df.loc[df["followers"] >= high].user_id.unique()

    return unsuccessful_creator_ids, successful_creator_ids


In [12]:
unsuccessful_ids, successful_ids = successful_creators_followers(follows_received, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = creators)


2016-05-30 00:00:00
subsetting...
High influencer boundary: 81.0
Low influencer boundary: 13.0


## Filter only actions that were sent to non-fans

In [13]:
table_data = pd.read_csv('user_types_ids.csv')

hermit_ids = table_data.loc[table_data.Type =='Hermit'].user_id.unique()
w_a_ids = table_data.loc[table_data.Type =='w_a'].user_id.unique()
f_a_ids = table_data.loc[table_data.Type =='f_a'].user_id.unique()
observer_ids = table_data.loc[table_data.Type =='Observer'].user_id.unique()

We merge the `actions_sent` dataset with a table containing the date each fan started following the creator.

In [14]:
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follows_received[["fan_id", "user_id", "date_sent"]]
followers.columns = ["fan_id", "user_id", "follower_since"]

actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

actions_sent['user_type'] = actions_sent.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
actions_sent['creator_type'] = actions_sent.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

Since we are interested in acquisition campaings, we need to produce a dataset that exclude actions targetting fans.
We do that using filters based on the date of the action and the date that the user became a fan of the content creator. The resulting dataframe is named `actions_sent_non_fans`.

We then filter only actions that happened before the user follows the content creator.

In [15]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_fans =  actions_sent[mask]
actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_25252/3855957735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))


## Non-follow Actions level

The activity level is defined as the number of actions performed by users. It is important to notice that we only observe actions targeting the 35k users that joined in march 2012. We consider this measure a proxy for the real activity level.

Let's begin by creating a dataset with all action received by those 35k users.

In [16]:
comments_received_35k = import_dta(path_dir, "12sample_comments_received.dta");
shares_received_35k = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received_35k = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received_35k = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received_35k:
        shares_received_35k.drop(columns=["song_id"])
shares_received_35k = shares_received_35k[['reposter_id', "owner_id", 'created_at']]
shares_received_35k['outbound_activity'] = 'share'
shares_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in likes_received_35k:
        likes_received_35k = likes_received_35k.drop(columns=["track_id"])
likes_received_35k['outbound_activity'] = 'like'
likes_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in comments_received_35k:
        comments_received_35k = comments_received_35k.drop(columns=["track_id"])
comments_received_35k['outbound_activity'] = 'comment'
comments_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

messages_received_35k["outbound_activity"] = 'message'
messages_received_35k.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

user_activity_data_35k = pd.concat([shares_received_35k, likes_received_35k, comments_received_35k, messages_received_35k])

user_activity_data_35k['day_yr_date'] = user_activity_data_35k.date_sent.dt.normalize()

reposts_df = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity == 'share']

%%%%%%%%%% 12sample_comments_received.dta %%%%%%%%%%
(21386, 4)
%%%%%%%%%% 12sample_reposts_received.dta %%%%%%%%%%
(83013, 4)
%%%%%%%%%% 12sample_favoritings_received.dta %%%%%%%%%%
(286903, 4)
%%%%%%%%%% 12sample_messages_received.dta %%%%%%%%%%
(17364, 3)


In [17]:
# path_dir_2 = r'/Users/../Volumes/Alter_outbound_activities/'

# comments_received_c = import_dta(path_dir_2, "12sample_1st_deg_comments_made.dta");
# shares_received_c = import_dta(path_dir_2, "12sample_1st_deg_reposts_made.dta");
# likes_received_c = import_dta(path_dir_2, "12sample_1st_deg_favoritings_made.dta");
# messages_received_c = import_dta(path_dir_2, "12sample_1st_deg_messages_sent.dta");

In [18]:
# if 'song_id' in shares_received_c:
#         shares_received_c.drop(columns=["song_id"])
# shares_received_c = shares_received_c[['reposter_id', "owner_id", 'created_at']]
# shares_received_c['inbound_activity'] = 'share'
# shares_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

# if 'track_id' in likes_received_c:
#         likes_received_c = likes_received_c.drop(columns=["track_id"])
# likes_received_c['inbound_activity'] = 'like'
# likes_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

# if 'track_id' in comments_received_c:
#         comments_received_c = comments_received_c.drop(columns=["track_id"])
# comments_received_c['inbound_activity'] = 'comment'
# comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

# messages_received_c["outbound_activity"] = 'message'
# messages_received_c.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

In [19]:
#user_activity_data_c = pd.concat([shares_received_c, likes_received_c, comments_received_c, messages_received_c])

Once more we create an object containing the unique ids of users in the resulting dataset. This will be used in a flow-chart, as explained.

# Priori response probabilities

In [20]:
import datetime
# Target Creation
delta = datetime.timedelta(days = days_delta)
mask = (actions_sent_non_fans['follower_since'] <= (actions_sent_non_fans['date_sent'] + delta))

In [21]:
def calcs():
    delta = datetime.timedelta(days = days_delta)
    mask = (actions_sent_non_fans['follower_since'] <= (actions_sent_non_fans['date_sent'] + delta))

    response_df = actions_sent_non_fans.copy()
    response_df.loc[mask, 'reward'] = 1
    mask = response_df['reward'].isnull()
    response_df.loc[mask, 'reward'] = 0

    mask = (actions_sent.date_sent >= actions_sent.follower_since)
    actions_sent_to_fans = actions_sent[mask]
    actions_sent_to_fans['week_yr_date'] = actions_sent_to_fans.week_yr.apply(lambda x: process_date(x))

    repost_prob_df = actions_sent_to_fans # filter for follower only
    repost_prob_df.sort_values(['user_id', 'fan_id'], inplace = True)
    repost_prob_df['reward_repost'] = np.nan
    for user_id in tqdm(repost_prob_df.user_id.unique()):
        for fan_id in repost_prob_df.loc[repost_prob_df.user_id == user_id].fan_id.unique():
            repost_prob_df.loc[(repost_prob_df.user_id == user_id)&
                                  (repost_prob_df.fan_id == fan_id),'reward_repost'] =\
            repost_prob_df.loc[(repost_prob_df.user_id == user_id)&
                                  (repost_prob_df.fan_id == fan_id)].date_sent.apply(
            lambda x : 1 if 
        (reposts_df.loc[
        (reposts_df.user_id == user_id)&
        (reposts_df.fan_id == fan_id)&
        (reposts_df.day_yr_date > x)&   
        (reposts_df.day_yr_date <= x + datetime.timedelta(days=days_delta))]).shape[0]>0 else 0)

    repost_prob_df.sort_values(by='date_sent', inplace = True)

    return response_df, repost_prob_df

In [22]:
response_df, repost_prob_df = calcs()

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_25252/3292797339.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_to_fans['week_yr_date'] = actions_sent_to_fans.week_yr.apply(lambda x: process_date(x))
/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_25252/3292797339.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repost_prob_df.sort_values(['user_id', 'fan_id'], inplace = True)
/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_25252/3292797339.py:16: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [23]:
analysis_data = pd.read_csv('indegree_level.csv', index_col = 0)
analysis_data.columns = ['fan_id', 'followers', 'outdegree_level', 'activities_performed',
       'activity_level', 'Type', 'contact_id', 'indegree', 'indegree_level']
response_df = response_df.merge(analysis_data, left_on = 'fan_id', right_on = 'fan_id', how = 'left')
repost_prob_df = repost_prob_df.merge(analysis_data, left_on = 'fan_id', right_on = 'fan_id', how = 'left')

In [24]:
#add indegree level

attempts = response_df.groupby(['creator_type', 'user_type', 'indegree_level'
                               ], as_index = False).size()
rewards = response_df.groupby(['creator_type', 'user_type' , 'indegree_level'
                              ], as_index = False).agg({'reward' : 'sum'})

priori_prob_df = rewards.merge(attempts)
priori_prob_df['P_resp_prob'] = priori_prob_df['reward']/priori_prob_df['size']
priori_prob_df 

,creator_type,user_type,indegree_level,reward,size,P_resp_prob
0,other,Hermit,high,1.0,3424,0.000292
1,other,Hermit,low,86.0,3226,0.026658
2,other,Hermit,other,73.0,7375,0.009898
3,other,Observer,high,3.0,155,0.019355
4,other,Observer,low,0.0,8,0.000000
5,other,Observer,other,9.0,140,0.064286
6,other,f_a,high,0.0,14340,0.000000
7,other,f_a,low,5.0,122,0.040984
8,other,f_a,other,14.0,316,0.044304
9,other,other,high,555.0,55994,0.009912


In [25]:
df = priori_prob_df 

pivoted_df = round(df.pivot(index='creator_type', columns=['user_type', 'indegree_level'], values=['P_resp_prob']),3)

# Display the pivoted DataFrame
pivoted_df

P_resp_prob                                                     \
user_type           Hermit               Observer                  f_a          
indegree_level        high    low  other     high    low  other   high    low   
creator_type                                                                    
other                  0.0  0.027  0.010    0.019  0.000  0.064  0.000  0.041   
successful             0.0  0.026  0.013    0.089  0.042  0.092  0.001  0.047   
unsuccessful           0.0  0.044  0.015    0.000  0.000  0.065  0.000  0.214   

                                                                 
user_type              other                  w_a                
indegree_level  other   high    low  other   high    low  other  
creator_type                                                     
other           0.044  0.010  0.116  0.122  0.035  0.000  0.227  
successful      0.031  0.045  0.072  0.141  0.116  0.176  0.203  
unsuccessful    0.098  0.003  0.160  0.128  0.018    NaN  0.217

In [27]:
#add indegree level

attempts = df.groupby(['creator_type', 'user_type','indegree_level'
                                  ], as_index = False).size()
rewards = repost_prob_df.groupby(['creator_type', 'user_type', 'indegree_level'
                                 ], as_index = False).agg({'reward_repost' : 'sum'})

priori_rep_df = rewards.merge(attempts)
priori_rep_df['P_rep_prob'] = priori_rep_df['reward_repost']/priori_rep_df['size'] 

In [28]:
df = priori_rep_df 

pivoted_df = round(df.pivot(index='creator_type', columns=['user_type', 'indegree_level'], values=['P_rep_prob']),3)

# Display the pivoted DataFrame
pivoted_df

P_rep_prob                                                  \
user_type          Hermit            Observer              f_a              
indegree_level       high  low other     high  low other  high  low other   
creator_type                                                                
other                 0.0  1.0   0.0      0.0  0.0   0.0   0.0  0.0   0.0   
successful            0.0  0.0   0.0      0.0  0.0   0.0  10.0  0.0   3.0   
unsuccessful          NaN  0.0   0.0      0.0  0.0   0.0   NaN  0.0   0.0   

                                                    
user_type      other                w_a             
indegree_level  high   low other   high  low other  
creator_type                                        
other            1.0  23.0  53.0    8.0  0.0   9.0  
successful      37.0  13.0  50.0  350.0  0.0  22.0  
unsuccessful     0.0   3.0   9.0    1.0  NaN   1.0

In [29]:
repost_prob_df.to_csv("../dir_indir_effects_tables/indir_effect_repost_granular.csv")
response_df.to_csv("../dir_indir_effects_tables/dir_effect_granular.csv")

### Redo calculations from MKTSC

In [30]:
#Indirect Returns
#P(focal follower making a repost) 

attempts = repost_prob_df.groupby(['user_type'
                                   , 'indegree_level'
                                  ], as_index = False).size()
rewards = repost_prob_df.groupby(['user_type'
                                  , 'indegree_level'
                                 ], as_index = False).agg({'reward_repost' : 'sum'})

priori_rep_df = rewards.merge(attempts)
priori_rep_df['P_rep_prob'] = priori_rep_df['reward_repost']/priori_rep_df['size'] 

In [31]:
df = priori_rep_df 

pivoted_df = round(df.pivot(index='user_type', columns=['indegree_level'], values=['P_rep_prob']),4)

# Display the pivoted DataFrame
pivoted_df.T

user_type                  Hermit  Observer     f_a   other     w_a
           indegree_level                                          
P_rep_prob high            0.0000       0.0  0.1282  0.0052  0.0283
           low             0.0011       0.0  0.0000  0.0081  0.0000
           other           0.0000       0.0  0.0238  0.0074  0.0165

In [32]:
#Direct Returns
#P(follower of focal following creator l)

attempts = response_df.groupby(['user_type', 'indegree_level'], as_index = False).size()
rewards = response_df.groupby(['user_type', 'indegree_level'], as_index = False).agg({'reward' : 'sum'})

priori_prob_df = rewards.merge(attempts)
priori_prob_df['P_resp_prob'] = priori_prob_df['reward']/priori_prob_df['size']

In [33]:
df = priori_prob_df[priori_prob_df.user_type!='other']

pivoted_df = round(df.pivot(index='user_type', columns=['indegree_level'], values=['P_resp_prob']),3)
# Display the pivoted DataFrame
pivoted_df.T

user_type                   Hermit  Observer    f_a    w_a
            indegree_level                                
P_resp_prob high             0.000     0.063  0.000  0.084
            low              0.030     0.034  0.053  0.120
            other            0.011     0.082  0.040  0.207

think about the shares unprovoked:
    share given action is 0 for low, but spontaneous shares might be a thing, we need to quantify them:
    shares per following per user type and indegree level

In [34]:
priori_rep_df

,user_type,indegree_level,reward_repost,size,P_rep_prob
0,Hermit,high,0.0,7,0.000000
1,Hermit,low,1.0,932,0.001073
2,Hermit,other,0.0,237,0.000000
3,Observer,high,0.0,190,0.000000
4,Observer,low,0.0,47,0.000000
5,Observer,other,0.0,208,0.000000
6,f_a,high,10.0,78,0.128205
7,f_a,low,0.0,86,0.000000
8,f_a,other,3.0,126,0.023810
9,other,high,38.0,7338,0.005179


In [35]:
repost_prob_df.groupby(['user_type','indegree_level']).size()

user_type  indegree_level
Hermit     high                  7
           low                 932
           other               237
Observer   high                190
           low                  47
           other               208
f_a        high                 78
           low                  86
           other               126
other      high               7338
           low                4787
           other             15052
w_a        high              12694
           low                   6
           other              1934
dtype: int64

In [36]:
80/(80+83+127)

0.27586206896551724

In [37]:
127/(80+83+127)

0.4379310344827586

In [38]:
#Indirect Returns
#P(focal follower making a repost) 

attempts = repost_prob_df.groupby(['user_type'
                                   #, 'indegree_level'
                                  ], as_index = False).size()
rewards = repost_prob_df.groupby(['user_type'
                                  #, 'indegree_level'
                                 ], as_index = False).agg({'reward_repost' : 'sum'})

priori_rep_df = rewards.merge(attempts)
priori_rep_df['P_rep_prob'] = priori_rep_df['reward_repost']/priori_rep_df['size'] 

priori_rep_df 

,user_type,reward_repost,size,P_rep_prob
0,Hermit,1.0,1179,0.000848
1,Observer,0.0,445,0.000000
2,f_a,13.0,290,0.044828
3,other,189.0,28221,0.006697
4,w_a,391.0,14634,0.026719


In [39]:
#Indirect Returns
#P(focal follower making a repost) 

attempts = repost_prob_df.groupby(['user_type'
                                   , 'indegree_level'
                                  ], as_index = False).size()
rewards = repost_prob_df.groupby(['user_type'
                                  , 'indegree_level'
                                 ], as_index = False).agg({'reward_repost' : 'sum'})

priori_rep_df = rewards.merge(attempts)
priori_rep_df['P_rep_prob'] = priori_rep_df['reward_repost']/priori_rep_df['size'] 

priori_rep_df 

,user_type,indegree_level,reward_repost,size,P_rep_prob
0,Hermit,high,0.0,7,0.000000
1,Hermit,low,1.0,932,0.001073
2,Hermit,other,0.0,237,0.000000
3,Observer,high,0.0,190,0.000000
4,Observer,low,0.0,47,0.000000
5,Observer,other,0.0,208,0.000000
6,f_a,high,10.0,78,0.128205
7,f_a,low,0.0,86,0.000000
8,f_a,other,3.0,126,0.023810
9,other,high,38.0,7338,0.005179


In [ ]:
(80*.125+0*83+0.0236*127)/(80+83+127)

In [41]:
max(analysis_data.indegree)

8440637